**Dự báo xu hướng giá CK bằng thuật toán Rừng ngẫu nhiên**

1. Tải dữ liệu
2. Tạo biến Y=0 nếu giá CK giảm và biến Y=1 nếu giá CK tăng.
3. Sử dụng hàm RandomForestRegressor từ thư viện sklearn.
4. Đánh giá độ chính xác của mô hình thông qua MSE.

**VCB**

In [1]:
import pandas as pd
# Đọc dữ liệu từ file CSV (có thể thay bằng file của bạn)
file_path = r'D:\DeTaiNam2024\SolieuHong\VCB.csv'
df = pd.read_csv(file_path)
# Kiểm tra tiêu đề các cột
df.head()


,Unnamed: 0,time,open,high,low,close,volume
0,0,2015-01-05,14.03,14.21,13.90,14.12,310010
1,1,2015-01-06,14.12,15.05,14.12,15.05,1684600
2,2,2015-01-07,15.09,15.31,14.65,14.87,1430340
3,3,2015-01-08,14.92,15.49,14.87,15.31,904240
4,4,2015-01-09,15.45,16.38,15.45,16.33,2286680


In [8]:
# Sắp xếp theo ngày
df = df.sort_values('time')

# Tạo cột Y: nếu giá đóng cửa hôm nay thấp hơn hôm qua, Y=0, nếu ngược lại Y=1
df['Y'] = (df['close'].diff() > 0).astype(int)

# Hiển thị dữ liệu với cột Y
print(df[['time', 'close', 'Y']])

            time  close  Y
0     2015-01-05  14.12  0
1     2015-01-06  15.05  1
2     2015-01-07  14.87  0
3     2015-01-08  15.31  1
4     2015-01-09  16.33  1
...          ...    ... ..
2421  2024-09-13  89.90  1
2422  2024-09-16  88.90  0
2423  2024-09-17  90.50  1
2424  2024-09-18  91.00  1
2425  2024-09-19  91.60  1

[2426 rows x 3 columns]


In [9]:
# Lấy đặc trưng (features) và biến mục tiêu (target)
X = df[['open', 'high', 'low', 'volume']]  # Các cột đặc trưng
Y= df['close']  # Biến mục tiêu cho hồi quy

In [10]:
from sklearn.model_selection import train_test_split # tải hàm train_test_split
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import  mean_squared_error, r2_score
import time
# Bắt đầu tính thời gian huấn luyện
start_time = time.time()
# Khởi tạo mô hình Rừng ngẫu nhiên cho phân loại
rf = RandomForestRegressor(n_estimators=100, random_state=42)
# Huấn luyện mô hình
rf.fit(X_train, Y_train)
# Kết thúc tính thời gian huấn luyện
train_time_cls = time.time() - start_time
print(f'Thời gian huấn luyện mô hình phân loại: {train_time_cls:.4f} giây')

# Bắt đầu tính thời gian dự đoán
start_time = time.time()

# Dự đoán trên tập kiểm tra
Y_pred = rf.predict(X_test)
# Kết thúc tính thời gian dự đoán
predict_time_cls = time.time() - start_time
print(f'Thời gian dự đoán mô hình phân loại: {predict_time_cls:.4f} giây')
# Đánh giá mô hình phân loại bằng MSE
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'R-squared (R2 Score): {r2:.2f}')

# In ra kết quả dự đoán và giá trị thực tế
results = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred})
print(results)


**Chạy tất cả chứng khoán trong folder**

In [9]:
import os
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report

In [18]:
def process_stock_file(file_path):
    # Đọc dữ liệu từ file csv
    df = pd.read_csv(file_path)
    
    # Nếu không đủ dữ liệu, bỏ qua file
    if df.shape[1] < 5:
        return None

    # Tạo các đặc trưng (features) và biến mục tiêu
    X = df[['open', 'high', 'low', 'volume']]
    Y = (df['close'] > df['open']).astype(int)  # Phân loại: xu hướng tăng (1) hoặc giảm (0)

    # Chia dữ liệu cho hồi quy và phân loại
    X_train, X_test,Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    
    # Biến lưu kết quả
    results = {}
    results['file'] = os.path.basename(file_path)

    # Huấn luyện mô hình Rừng ngẫu nhiên cho hồi quy
    start_time = time.time()
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, Y_train)
    train_time = time.time() - start_time

    #Dự báo
    start_time = time.time()
    Y_pred = rf.predict(X_test)
    predict_time = time.time() - start_time

     # Tính độ chính xác
    mse = mean_squared_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)
    results['MSE'] = mse
    results['R2'] = r2
    return results


In [12]:
def all_stocks(folder, output):
    # Danh sách lưu trữ kết quả cho mỗi file
    all_results = []

    # Duyệt qua từng file trong thư mục
    for file_name in os.listdir(folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder, file_name)
            print(f'Đang xử lý {file_name}...')
            result = process_stock_file(file_path)
            if result:
                all_results.append(result)

    # Chuyển danh sách kết quả thành DataFrame
    results_df = pd.DataFrame(all_results)

    # Ghi kết quả ra file CSV
    results_df.to_csv(output, index=False)
    print(f'Kết quả đã được ghi ra {output}')


In [19]:
# Đường dẫn đến thư mục chứa các file chứng khoán
folder = r'D:\DeTaiNam2024\SoLieuHong'

# Đường dẫn đến file kết quả output
output = r'D:\DeTaiNam2024\SoLieuHong\output\RandomForest_Regression_stock_results1.csv'

# Chạy xử lý tất cả các file trong thư mục và lưu kết quả
all_stocks(folder, output)

Đang xử lý acb.csv...
Đang xử lý DHG.csv...
Đang xử lý FPT.csv...
Đang xử lý HAP.csv...
Đang xử lý HNX.csv...
Đang xử lý HNX30.csv...
Đang xử lý HPG.csv...
Đang xử lý KDC.csv...
Đang xử lý REE.csv...
Đang xử lý SAM.csv...
Đang xử lý TMS.csv...
Đang xử lý VCB.csv...
Đang xử lý VIC.csv...
Đang xử lý VN30.csv...
Đang xử lý VNINDEX.csv...
Đang xử lý VNM.csv...
Kết quả đã được ghi ra D:\DeTaiNam2024\SoLieuHong\output\RandomForest_Regression_stock_results1.csv


**FPT**

In [22]:
import pandas as pd
# Đọc dữ liệu từ file CSV (có thể thay bằng file của bạn)
file_path = r'D:\DeTaiNam2024\SolieuHong\VCB.csv'
df = pd.read_csv(file_path)
# Kiểm tra tiêu đề các cột
df.head()


,Unnamed: 0,time,open,high,low,close,volume
0,0,2015-01-05,14.03,14.21,13.90,14.12,310010
1,1,2015-01-06,14.12,15.05,14.12,15.05,1684600
2,2,2015-01-07,15.09,15.31,14.65,14.87,1430340
3,3,2015-01-08,14.92,15.49,14.87,15.31,904240
4,4,2015-01-09,15.45,16.38,15.45,16.33,2286680


In [28]:
# Sắp xếp theo ngày
df = df.sort_values('time')

# Tạo cột Y: nếu giá đóng cửa hôm nay thấp hơn hôm qua, Y=0, nếu ngược lại Y=1
df['Y'] = (df['close'].diff() > 0).astype(int)

# Hiển thị dữ liệu với cột Y
print(df[['time', 'close', 'Y']])

            time  close  Y
0     2015-01-05  14.12  0
1     2015-01-06  15.05  1
2     2015-01-07  14.87  0
3     2015-01-08  15.31  1
4     2015-01-09  16.33  1
...          ...    ... ..
2421  2024-09-13  89.90  1
2422  2024-09-16  88.90  0
2423  2024-09-17  90.50  1
2424  2024-09-18  91.00  1
2425  2024-09-19  91.60  1

[2426 rows x 3 columns]


In [29]:
# Lấy đặc trưng (features) và biến mục tiêu (target)
X = df[['open', 'high', 'low', 'volume']]  # Các cột đặc trưng
Y= df['close']  # Biến mục tiêu cho hồi quy

In [30]:
from sklearn.model_selection import train_test_split # tải hàm train_test_split
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [34]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import  mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score



import matplotlib.pyplot as plt
import time
# Bắt đầu tính thời gian huấn luyện
start_time = time.time()
# Khởi tạo mô hình Rừng ngẫu nhiên cho phân loại
rf = RandomForestRegressor(n_estimators=100, random_state=42)
scores = cross_val_score(rf, X, Y, cv=5, scoring='r2')
print(f'Cross-validated R^2 scores: {scores}')
print(f'Average R^2 score: {scores.mean()}')
# Huấn luyện mô hình
rf.fit(X_train, Y_train)
# Kết thúc tính thời gian huấn luyện
train_time_cls = time.time() - start_time
print(f'Thời gian huấn luyện mô hình phân loại: {train_time_cls:.4f} giây')

# Bắt đầu tính thời gian dự đoán
start_time = time.time()

# Dự đoán trên tập kiểm tra
Y_pred = rf.predict(X_test)
# Kết thúc tính thời gian dự đoán
predict_time_cls = time.time() - start_time
print(f'Thời gian dự đoán mô hình phân loại: {predict_time_cls:.4f} giây')
# Đánh giá mô hình phân loại bằng MSE
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'R-squared (R2 Score): {r2:.2f}')

# In ra kết quả dự đoán và giá trị thực tế
results = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred})
print(results)


Cross-validated R^2 scores: [-0.21156941  0.98920766  0.98396105  0.96398241  0.31177795]
Average R^2 score: 0.6074719341361403
Thời gian huấn luyện mô hình phân loại: 30.1030 giây
Thời gian dự đoán mô hình phân loại: 0.0313 giây
Mean Squared Error (MSE): 0.30
R-squared (R2 Score): 1.00
      Actual  Predicted
1491   64.31    63.6710
387    24.65    24.1713
1192   54.78    55.2995
410    26.00    26.6560
482    21.80    21.8675
...      ...        ...
679    23.36    23.3769
178    19.90    19.8156
1011   35.32    35.4997
324    20.80    21.1634
2201   85.80    85.3271

[486 rows x 2 columns]
